# Object Detection

IoU is calculated as the area of overlap between the two bounding boxes divided by the area of their union. It's a simple and effective metric, but it has a limitation: it doesn't take into account the relative positions of the boxes when they don't overlap. This means that two pairs of boxes can have the same IoU even if one pair is better aligned than the other. IoU ranges from 0 to 1, where 0 means no overlap and 1 means perfect overlap. However, IoU is not differentiable at 0, which makes it unsuitable for use as a loss function in a learning algorithm.

GIoU extends IoU by also considering the area of the smallest enclosing box that contains both boxes. Specifically, it subtracts from the IoU a term that represents the ratio of the area outside the union but inside the enclosing box to the area of the enclosing box. This makes GIoU a more comprehensive metric that can distinguish between different relative positions of the boxes. GIoU is differentiable and can be used as a loss function.


$$ IoU = \frac{b_1 \cap b_2}{b_1 \cup b_2} $$

$$ GIoU = IoU- \frac{C - (b_1 \cup b_2)}{C} $$

$$ DIoU = IoU - \frac{d^2}{c^2} $$

$$ CIoU = IoU - \frac{d^2 + \rho^2}{c^2} $$

* $C$ is the area of the smallest enclosing box that contains both bounding boxes.
* $d$ is the Euclidean distance between the central points of the two bounding boxes.
* $c$ is the diagonal length of the smallest enclosing box that contains both bounding boxes.
* $\rho$ is the difference in aspect ratio between the two bounding boxes.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np


class Box:
    def __init__(self, x, y, w, h):
        self.x = x
        self.y = y
        self.w = w
        self.h = h

    def area(self):
        return self.w * self.h

    def center(self):
        return self.x + self.w / 2, self.y + self.h / 2

    def dist(self, other):
        return np.sqrt((self.center()[0] - other.center()[0])**2 + (self.center()[1] - other.center()[1])**2)

    def area_of_union(self, other):
        inter_box = self.intersection_box(other)
        union = self.area() + other.area()
        if inter_box is None:
            return union
        return union - inter_box.area()

    def enclosing_box(self, other):
        x = min(self.x, other.x)
        y = min(self.y, other.y)
        w = max(self.x + self.w, other.x + other.w) - x
        h = max(self.y + self.h, other.y + other.h) - y
        return Box(x, y, w, h)

    def intersection_box(self, other):
        x = max(self.x, other.x)
        y = max(self.y, other.y)
        w = min(self.x + self.w, other.x + other.w) - x
        h = min(self.y + self.h, other.y + other.h) - y
        if w <= 0 or h <= 0:
            return None
        return Box(x, y, w, h)

    def iou(self, other):
        inter_box = self.intersection_box(other)
        if inter_box is None:
            return 0
        return inter_box.area() / self.area_of_union(other)

    def giou(self, other):
        union_area = self.area_of_union(other)
        enc_area = self.enclosing_box(other).area()
        return self.iou(other) - (enc_area - union_area) / enc_area

    def _diagonal_dist(self, other):
        start_x = min(self.x, other.x)
        start_y = min(self.y, other.y)
        end_x = max(self.x + self.w, other.x + other.w)
        end_y = max(self.y + self.h, other.y + other.h)
        return np.sqrt((end_x - start_x)**2 + (end_y - start_y)**2)

    def diou(self, other):
        return self.iou(other) - self.dist(other)**2 / self._diagonal_dist(other)**2

    def draw(self, color='r'):
        plt.gca().add_patch(patches.Rectangle((self.x, self.y),
                                              self.w, self.h, fill=None, alpha=1, edgecolor=color))

    def __str__(self):
        return f'Box({self.x}, {self.y}, {self.w}, {self.h}), area={self.area()}, center={self.center()}'

In [ ]:
def compare(box1, box2):
    intersection = b1.intersection_box(b2)
    enclosing = b1.enclosing_box(b2)
    print('Box1\t\t', b1)
    print('Box2\t\t', b2)
    print('Intersection\t', intersection)
    print('Enclosing\t', enclosing)
    print('Distance\t', b1.dist(b2))
    print('Union Area\t', b1.area_of_union(b2))
    print('IoU\t\t', b1.iou(b2))
    print('GIoU\t\t', b1.giou(b2))
    print('DIoU\t\t', b1.diou(b2))
    plt.figure()
    plt.legend(handles=[patches.Patch(color='red', label='Box1'), patches.Patch(color='green', label='Box2'), patches.Patch(
        color='black', label='Enclosing Box'), patches.Patch(color='blue', label='Intersection Box')])
    plt.xlim(0, 110)
    plt.ylim(0, 110)
    b1.draw('red')
    b2.draw('green')
    enclosing.draw('black')
    if intersection is not None:
        intersection.draw('blue')


b1 = Box(10, 10, 40, 40)
b2 = Box(20, 20, 40, 40)
compare(b1, b2)

In [ ]:
b1 = Box(10, 10, 40, 40)
b2 = Box(60, 60, 40, 40)
compare(b1, b2)